In [ ]:
print("The following is for exercise 4")

The following is for exercise 4


In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
import numpy
numpy.random.seed(1)
a = numpy.random.randn(128)

In [ ]:
# Convert values from double precision to single precision (requirement for CUDA)
a = a.astype(numpy.float32)

In [ ]:
# Allocate memory in device
a_gpu = cuda.mem_alloc(a.nbytes)

In [ ]:
# Copy array data from a to a_gpu (from host to device)
cuda.memcpy_htod(a_gpu, a)

In [ ]:
# Create CUDA kernel
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = blockIdx.x*blockDim.x*2 + threadIdx.x;
    int offset = blockDim.x*2;
    
    a[idx] *= 2;
    a[idx + offset] *= 2;
  }
  """)

In [ ]:
# Here we choose the block and grid dimensions
# TODO: add grid dimensions
grid_dim = (2,1)
# TODO: add block dimensions
block_dim = (32,1,1)
func = mod.get_function("doublify")
func(a_gpu, block=block_dim, grid = grid_dim)

In [ ]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled[:10])
print(a[:10])

[ 3.2486908  -1.2235128  -1.0563436  -2.1459372   1.7308153  -4.6030774
  3.4896235  -1.5224138   0.6380782  -0.49874076]
[ 1.6243454  -0.6117564  -0.5281718  -1.0729686   0.86540765 -2.3015387
  1.7448118  -0.7612069   0.3190391  -0.24937038]
